## Implementing Gradient Descent

Using the student admissions data in `binary.csv`.

In [1]:
import numpy as np
import pandas as pd

### Data Cleanup

The student admissions data in `binary.csv` has 4 fields: `admit`, `gre`, `gpa`, and `rank`. To cleanup the data we need to:

1. One-hot encode the rank data. Converting 1-4 value into 4 columns of 1 or 0.
2. Scale the `gre` and `gpa` data so that the sigmoid function will distribute the values reasonably. We'll scale the values to have a mean of 0 and standard deviation of 1.

In [2]:
# read in the data
admissions = pd.read_csv('binary.csv')
admissions.head()

,admit,gre,gpa,rank
0,0,380,3.61,3
1,1,660,3.67,3
2,1,800,4.00,1
3,1,640,3.19,4
4,0,520,2.93,4


In [12]:
# make dummy variables for rank
data = pd.concat([admissions, pd.get_dummies(admissions['rank'], prefix='rank',)], axis=1)
data = data.drop('rank', axis=1)
data.head()

,admit,gre,gpa,rank_1,rank_2,rank_3,rank_4
0,0,380,3.61,0,0,1,0
1,1,660,3.67,0,0,1,0
2,1,800,4.00,1,0,0,0
3,1,640,3.19,0,0,0,1
4,0,520,2.93,0,0,0,1


In [29]:
# scale the gre and gpa columns

for field in ['gre', 'gpa']:
    mean, std = data[field].mean(), data[field].std()    
    data.loc[:, field] = (data[field] - mean) / std

data.head()

,admit,gre,gpa,rank_1,rank_2,rank_3,rank_4
66,0,1.371118,0.626788,0,0,0,1
284,1,-1.298342,0.016415,0,1,0,0
110,0,0.837226,-0.806261,0,0,0,1
41,1,-0.052594,-0.169350,0,1,0,0
1,1,0.659262,0.759477,0,0,1,0


In [30]:
# split into 90% training and 10% testing groups

# get an array of indexes for the training data
sample = np.random.choice(data.index, size = int(len(data.index) * 0.9), replace = False)

# split into data and test buckets
data, test_data = data.loc[sample], data.drop(sample)

# split into features and targets
features, targets = data.drop('admit', axis=1), data['admit']
features_test, targets_test = test_data.drop('admit', axis=1), test_data['admit']

### Implement Training for the Admission Data

In [37]:
def sigmoid(x):
    """
    Calculate sigmoid
    """
    return 1 / (1 + np.exp(-x))

n_records, n_features = features.shape
last_loss = None

# Initialize weights
weights = np.random.normal(scale=1 / n_features**.5, size=n_features)

# Neural Network hyperparameters
epochs = 1000
learnrate = 0.5

for e in range(epochs):
    del_w = np.zeros(weights.shape)
    for x, y in zip(features.values, targets):
        # Loop through all records, x is the input, y is the target

        # Note: We haven't included the h variable from the previous
        #       lesson. You can add it if you want, or you can calculate
        #       the h together with the output

        # Calculate the output
        output = sigmoid(np.dot(x, weights))

        # Calculate the error
        error = y - output

        # Calculate the error term - use shortcut for f'(h) with sigmoid function
        error_term = error * output * (1 - output)

        # Calculate the change in weights for this sample
        #       and add it to the total weight change
        del_w += error_term * x

    # Update weights using the learning rate and the average change in weights
    weights += learnrate * del_w / n_records

    # Printing out the mean square error on the training set
    if e % (epochs / 10) == 0:
        out = sigmoid(np.dot(features, weights))
        loss = np.mean((out - targets) ** 2)
        if last_loss and last_loss < loss:
            print("Train loss: ", loss, "  WARNING - Loss Increasing")
        else:
            print("Train loss: ", loss)
        last_loss = loss


# Calculate accuracy on test data
tes_out = sigmoid(np.dot(features_test, weights))
predictions = tes_out > 0.5
accuracy = np.mean(predictions == targets_test)
print("Prediction accuracy: {:.3f}".format(accuracy))

Train loss:  0.2955585638399363
Train loss:  0.2127842170194329
Train loss:  0.1990512403238906
Train loss:  0.1945629646162711
Train loss:  0.19272884594138404
Train loss:  0.191860078331654
Train loss:  0.1914015007854246
Train loss:  0.19113795615258009
Train loss:  0.190975773645248
Train loss:  0.19087032433271872
Prediction accuracy: 0.533
